In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import json
import time
import os
import re
import requests

# 저장 폴더 설정
save_directory = "pdf_files"
os.makedirs(save_directory, exist_ok=True)

# JSON 파일 불러오기
with open("patent_cases_all_pages.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 크롬 드라이버 세팅
options = webdriver.ChromeOptions()
options.add_experimental_option("prefs", {
    "download.prompt_for_download": False,
    "plugins.always_open_pdf_externally": True
})
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

for item in data:
    title = item["제목"].strip()
    url = item["링크"]
    safe_title = re.sub(r'[\\/*?:"<>|]', '', title)

    print(f"📄 처리 중: {title}")

    try:
        driver.get(url)
        time.sleep(2)

        # 자바스크립트 기반 다운로드 링크 추출
        link = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "a[href^='javascript:download']"))
        )
        href = link.get_attribute("href")

        match = re.search(r"download\('(.+?)','(.+?)','.*?','.*?'\)", href)
        if match:
            file_key, file_id = match.groups()
            download_url = f"https://patent.scourt.go.kr/dcboard/new/DcNewsAttachFileDown.work?fileKey={file_key}&fileId={file_id}"

            response = requests.get(download_url)
            if response.status_code == 200:
                file_path = os.path.join(save_directory, f"{safe_title}.pdf")
                with open(file_path, "wb") as f:
                    f.write(response.content)
                print(f"✅ PDF 저장 완료: {safe_title}.pdf")
            else:
                print(f"❌ 다운로드 실패 (HTTP {response.status_code}): {title}")
        else:
            print(f"⚠️ JavaScript 링크 파싱 실패: {title}")

    except TimeoutException:
        print(f"❌ PDF 요소 대기 실패 (Timeout): {title}")
    except Exception as e:
        print(f"❌ 오류 발생: {e}")

# 크롬 드라이버 종료
driver.quit()
print("\n✅ 전체 PDF 다운로드 완료")


📄 처리 중: [상표]DoctorsNow 사건(특허법원 2024허15431)
⚠️ JavaScript 링크 파싱 실패: [상표]DoctorsNow 사건(특허법원 2024허15431)
📄 처리 중: [특허]수상 레저용 보트 권리범위확인 사건(특허법원 2024허14643)
⚠️ JavaScript 링크 파싱 실패: [특허]수상 레저용 보트 권리범위확인 사건(특허법원 2024허14643)
📄 처리 중: [특허]쇼케이스용 착탈식 자동 회전 도어 권리범위확인 사건(특허법원 2024허12685)
⚠️ JavaScript 링크 파싱 실패: [특허]쇼케이스용 착탈식 자동 회전 도어 권리범위확인 사건(특허법원 2024허12685)
📄 처리 중: [디자인]여성 원피스의 등록무효(디) 사건(특허법원 2024허13978)
⚠️ JavaScript 링크 파싱 실패: [디자인]여성 원피스의 등록무효(디) 사건(특허법원 2024허13978)
📄 처리 중: [상표]NK365 상표등록무효 사건(특허법원 2024허12272)


KeyboardInterrupt: 